# Loading the tests results
To start with, we download the breton test results.
1. load the api key and connect to the client
2. Find the tests collection and subcollection with a recursive function
3. Convert the documents in one big dictionary
4. Load the dictionary as a json file

In [ ]:
%pip install --quiet firebase-admin google-cloud-firestore


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
import os
from typing import Dict, Any, List
import firebase_admin  # Import the module directly
from firebase_admin import credentials, initialize_app
from google.cloud import firestore
import json
from datetime import datetime

# 1) Make sure the env var is set:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/Users/alan/Repos/analeksis/firebase-admin-key.json"
)

# 2) Initialize Admin SDK (safe to call once)
if not firebase_admin._apps:  # Check if any Firebase apps are initialized
    cred = credentials.ApplicationDefault()
    initialize_app(cred)

# 3) Firestore client
db = firestore.Client()  # Uses GOOGLE_APPLICATION_CREDENTIALS


def _convert_firestore_types(value):
    """
    Convert Firestore-specific types (Timestamp, DocumentReference, GeoPoint) to JSON-serializable forms.
    """
    from google.cloud.firestore_v1 import DocumentReference
    from google.cloud.firestore_v1._helpers import GeoPoint
    from google.protobuf.timestamp_pb2 import Timestamp as PbTimestamp

    # Firestore Timestamp comes in as 'datetime' when using google-cloud-firestore
    import datetime as _dt

    if isinstance(value, _dt.datetime):
        # ISO 8601 string
        return value.isoformat()
    if isinstance(value, DocumentReference):
        return {"__type__": "DocumentReference", "path": value.path}
    if isinstance(value, GeoPoint):
        return {
            "__type__": "GeoPoint",
            "latitude": value.latitude,
            "longitude": value.longitude,
        }
    if isinstance(value, (list, tuple)):
        return [_convert_firestore_types(v) for v in value]
    if isinstance(value, dict):
        return {k: _convert_firestore_types(v) for k, v in value.items()}
    return value


def doc_to_dict(doc_ref: firestore.DocumentReference) -> Dict[str, Any]:
    """
    Return a JSON-serializable dict for a document, including nested subcollections.
    Shape:
    {
        "id": "<doc_id>",
        "data": {...},
        "subcollections": {
            "<subcol_name>": [ {doc...}, ... ],
            ...
        }
    }
    """
    snap = doc_ref.get()
    data = snap.to_dict() or {}
    data = _convert_firestore_types(data)

    # Recurse into subcollections
    sub_map: Dict[str, List[Dict[str, Any]]] = {}
    for subcol in doc_ref.collections():
        docs_list = []
        for subdoc in subcol.stream():
            docs_list.append(doc_to_dict(subdoc.reference))
        sub_map[subcol.id] = docs_list

    return {"id": doc_ref.id, "data": data, "subcollections": sub_map}


# --- Export only "tests" collection ---
def export_tests_collection(limit=None):
    export = []
    col = db.collection("tests")
    count = 0
    for doc in col.stream():
        export.append(doc_to_dict(doc.reference))
        count += 1
        if limit and count >= limit:
            break
    return export


# ---- Run the export ----
export_dict = export_tests_collection()  # pass limit=NN for quick tests

# Write to a timestamped JSON file
outfile = f"all_tests_export_{datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')}.json"
with open(outfile, "w", encoding="utf-8") as f:
    json.dump(export_dict, f, ensure_ascii=False, indent=2)

outfile

/var/folders/f2/5mnmp0cs0bv_h2fdx27y8w580000gn/T/ipykernel_23601/618359319.py:97: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  outfile = f"all_tests_export_{datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')}.json"


'all_tests_export_20250907T173355Z.json'